<a href="https://colab.research.google.com/github/m-njera/crop_disease/blob/master/nasnet_mobile_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from glob import glob
from random import shuffle
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution1D, concatenate, SpatialDropout1D, GlobalMaxPool1D, GlobalAvgPool1D, Embedding, \
    Conv2D, SeparableConv1D, Add, BatchNormalization, Activation, GlobalAveragePooling2D, LeakyReLU, Flatten
from keras.layers import Dense, Input, Dropout, MaxPooling2D, Concatenate, GlobalMaxPooling2D, GlobalAveragePooling2D, \
    Lambda, Multiply, LSTM, Bidirectional, PReLU, MaxPooling1D
from keras.layers.pooling import _GlobalPooling1D
from keras.losses import mae, sparse_categorical_crossentropy, binary_crossentropy
from keras.models import Model
from keras.applications.nasnet import NASNetMobile, NASNetLarge, preprocess_input
from keras.optimizers import Adam, RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from imgaug import augmenters as iaa


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
base_dir = '/content/drive/My Drive/crop/'
train_data_dir = os.path.join(base_dir,'train')

image_width = 96
image_height = 96
batch_size=64

datagen = ImageDataGenerator(rescale=1./255,                                    
                                    height_shift_range=0.5,
                                    width_shift_range = 0.5,
                                    zoom_range = 0.3,
                                    
                                    validation_split=0.30)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width,image_height),
    batch_size=batch_size,
    shuffle=False,
    class_mode = 'categorical',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_width,image_height),
    shuffle=False,
    batch_size=batch_size,
    class_mode = 'categorical',
    subset='validation'
)


Found 614 images belonging to 3 classes.
Found 262 images belonging to 3 classes.


In [0]:
def get_model_classif_nasnet():
    inputs = Input((96, 96, 3))
    base_model = NASNetMobile(include_top=False, input_shape=(96, 96, 3), weights=None)
    x = base_model(inputs)
    out1 = GlobalMaxPooling2D()(x)
    out2 = GlobalAveragePooling2D()(x)
    out3 = Flatten()(x)
    out = Concatenate(axis=-1)([out1, out2, out3])
    out = Dropout(0.5)(out)
    out = Dense(3, activation="sigmoid", name="3_")(out)
    model = Model(inputs, out)
    model.compile(optimizer=Adam(0.0001), loss=binary_crossentropy, metrics=['acc'])
    model.summary()

    return model

In [0]:
model = get_model_classif_nasnet()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 96, 96, 3)    0                                            
__________________________________________________________________________________________________
NASNet (Model)                  (None, 3, 3, 1056)   4269716     input_2[0][0]                    
__________________________________________________________________________________________________
global_max_pooling2d_1 (GlobalM (None, 1056)         0           NASNet[1][0]                     
___________________________________________________________________

In [0]:
batch_size=32
h5_path = "model.h5"
checkpoint = ModelCheckpoint(h5_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# history = model.fit_generator(
#     data_gen(train, id_label_map, batch_size, augment=True),
#     validation_data=data_gen(val, id_label_map, batch_size),
#     epochs=2, verbose=1,
#     callbacks=[checkpoint],
#     steps_per_epoch=len(train) // batch_size,
#     validation_steps=len(val) // batch_size)
batch_size=64
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs=20, verbose=1,
    callbacks=[checkpoint],
    steps_per_epoch=50,
    validation_steps=50)

model.load_weights(h5_path)



Epoch 1/20
50/50 [==============================] - 1043s 21s/step - loss: 0.9006 - acc: 0.5933 - val_loss: 0.7129 - val_acc: 0.6330

Epoch 00001: val_acc improved from -inf to 0.63295, saving model to model.h5
Epoch 2/20
50/50 [==============================] - 799s 16s/step - loss: 0.7508 - acc: 0.6253 - val_loss: 0.7179 - val_acc: 0.6439

Epoch 00002: val_acc improved from 0.63295 to 0.64389, saving model to model.h5
Epoch 3/20
50/50 [==============================] - 811s 16s/step - loss: 0.7691 - acc: 0.6129 - val_loss: 0.7928 - val_acc: 0.6497

Epoch 00003: val_acc improved from 0.64389 to 0.64975, saving model to model.h5
Epoch 4/20
50/50 [==============================] - 807s 16s/step - loss: 0.7807 - acc: 0.5919 - val_loss: 0.7501 - val_acc: 0.6573

Epoch 00004: val_acc improved from 0.64975 to 0.65725, saving model to model.h5
Epoch 5/20
50/50 [==============================] - 806s 16s/step - loss: 0.7594 - acc: 0.6088 - val_loss: 0.8950 - val_acc: 0.6496

Epoch 00005: va

KeyboardInterrupt: ignored

In [0]:
preds = []
ids = []

In [0]:
for batch in chunker(test_files, batch_size):
    X = [preprocess_input(cv2.imread(x)) for x in batch]
    ids_batch = [get_id_from_file_path(x) for x in batch]
    X = np.array(X)
    preds_batch = ((model.predict(X).ravel()*model.predict(X[:, ::-1, :, :]).ravel()*model.predict(X[:, ::-1, ::-1, :]).ravel()*model.predict(X[:, :, ::-1, :]).ravel())**0.25).tolist()
    preds += preds_batch
    ids += ids_batch

In [0]:
df = pd.DataFrame({'id':ids, 'label':preds})
df.to_csv("baseline_nasnet.csv", index=False)
df.head()